In [1]:
from dotenv import load_dotenv

In [6]:
from langchain_openai import ChatOpenAI

In [2]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import OpenAI
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool

/Users/ali/anaconda3/envs/graph_rag/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
from dotenv import load_dotenv

In [4]:
load_dotenv()

True

In [7]:
llm = ChatOpenAI()

In [5]:
state = {}

In [8]:
from langchain_core.prompts import ChatPromptTemplate

In [17]:
def get_data(state,topic):
    """
    A function to generate data;
    """
    state['topic'] = topic
    prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a data generator agent that get topic from a user and generate a detail about it.",
        ),
        ("human", "Generate a description for {topic}"),
    ]
    )

    chain = prompt | llm
    ai_msg = chain.invoke({"topic":topic})
    return ai_msg.content


In [18]:
description = get_data(state,"Neural Network")

In [19]:
state['data'] = description

In [20]:
state

{'data': 'Neural networks are a type of artificial intelligence that are designed to mimic the way the human brain operates. They consist of interconnected nodes, or artificial neurons, which work together to process complex information and make decisions. Neural networks are capable of learning from data, which allows them to recognize patterns, make predictions, and solve difficult problems. They are used in a wide range of applications, including image and speech recognition, natural language processing, and autonomous vehicles. Neural networks have become increasingly popular in recent years due to their ability to perform tasks that were previously thought to be impossible for computers.',
 'topic': 'Neural Network'}

In [30]:
from langchain.output_parsers import PydanticOutputParser

In [29]:
# Here's another example, but with a compound typed field.

    

In [40]:
def scenario_generator(state):
    """
    An Experience Scenario maker that design scenario for interview.
    """
    class Scenario(BaseModel):
        scenario: str = Field(description="The generated scenario for paticular topic")
    prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a professional examiner and interviewer. You will make a detail secenario"
            "The scenario will explain the real world sitation about given topic and base generated data"
            "Please keep scenario concise and short like upto 3 sentences"
            "You will return scenario in json with key scenario"
        ),
        ("human", "Generate realworld scenario for exam on {topic} with base data {data}"),
    ]
    )
    parser = PydanticOutputParser(pydantic_object=Scenario) 
    chain = prompt | llm | parser
    ai_msg = chain.invoke({"topic":state['topic'],'data':state['data']})
    state['scenario'] = ai_msg.scenario
    return ai_msg


In [37]:
scenario = scenario_generator(state)

In [47]:
def question_generator(state):
    """
    You are an examiner and you need evaluate a candiate 's knowledge of a particular subject.
    You will ask questions based on given scenario, and data. 
    Also you will have access to previous asked questions. There might be previous asked questions.
    You will not answer question your self. You also will not help user in question.
    """

    state['questions'] = state.get('questions',[])
    class QuestionObj(BaseModel):
        question: str = Field(description="The generated question for paticular topic")
    
    prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a professional examiner and interviewer. You will ask question one at a time."
            "You will ask questions based on given scenario, and data."
            "Also you will have access to previous asked questions. There might be previous asked questions."
            "You will not answer question your self. You also will not help user in question."
            "You will create a json output with key question"
        ),
        ("human", "Generate realworld scenario for exam on {topic} with base data {data} and previous questions  {previous_questions}"),
    ]
    )
    parser = PydanticOutputParser(pydantic_object=QuestionObj) 
    chain = prompt | llm | parser
    ai_msg = chain.invoke({"topic":state['topic'],'data':state['data'],'previous_questions':state['questions']})
    state['questions'].append(ai_msg.question)
    return ai_msg

In [48]:
question_generator(state)

QuestionObj(question='How can neural networks be used in image recognition applications?')

In [49]:
state

{'data': 'Neural networks are a type of artificial intelligence that are designed to mimic the way the human brain operates. They consist of interconnected nodes, or artificial neurons, which work together to process complex information and make decisions. Neural networks are capable of learning from data, which allows them to recognize patterns, make predictions, and solve difficult problems. They are used in a wide range of applications, including image and speech recognition, natural language processing, and autonomous vehicles. Neural networks have become increasingly popular in recent years due to their ability to perform tasks that were previously thought to be impossible for computers.',
 'topic': 'Neural Network',
 'scenario': 'A tech company is developing a new neural network model for autonomous vehicles. The neural network is trained on a large dataset of images and sensor data collected from various driving scenarios. The company is conducting exams to evaluate the performa

In [50]:
question_generator(state)

QuestionObj(question='What role do neural networks play in natural language processing applications?')

In [51]:
state

{'data': 'Neural networks are a type of artificial intelligence that are designed to mimic the way the human brain operates. They consist of interconnected nodes, or artificial neurons, which work together to process complex information and make decisions. Neural networks are capable of learning from data, which allows them to recognize patterns, make predictions, and solve difficult problems. They are used in a wide range of applications, including image and speech recognition, natural language processing, and autonomous vehicles. Neural networks have become increasingly popular in recent years due to their ability to perform tasks that were previously thought to be impossible for computers.',
 'topic': 'Neural Network',
 'scenario': 'A tech company is developing a new neural network model for autonomous vehicles. The neural network is trained on a large dataset of images and sensor data collected from various driving scenarios. The company is conducting exams to evaluate the performa

In [82]:
from typing import Literal
def router_node(user_input,state):
    """
    You are a router node that need to decide  which path to take.
    """
    
    state['questions'] = state.get('',[])
    class PathObj(BaseModel):
        next_node: Literal['QUESTION_NODE','LLM_NODE','CLARIFICATION_NODE'] = Field(default="QUESTION_NODE", description="Which path to take")
    
    prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are router node. You have a list of path and you need to decide which path to follows."
            "You have acces to questions. If user response is related particular to question that is mentioned go to QUESTION_NODE"
            "If user response is general then go to LLM Node"
            "If user is asking for some clarification then go to  CLARIFICATION_NODE"
            "The available options are ['QUESTION_NODE','LLM_NODE','CLARIFICATION_NODE']"
            "You will create a json output with key next_node"
        ),
        # ("human", "Generate realworld scenario for exam on {topic} with base data {data} and previous questions  {previous_questions}"),
        ("human", "What will be next state {input}, Available options  are ['QUESTION_NODE','LLM_NODE','CLARIFICATION_NODE']. This is list of given question {question}"),
    ]
    )
    parser = PydanticOutputParser(pydantic_object=PathObj) 
    chain = prompt | llm | parser
    given_question = state['questions'][-1] if len(state['questions']) > 0 else []
    ai_msg = chain.invoke({"input":user_input,'question':given_question})
    return ai_msg.next_node

In [83]:
state

{'data': 'Neural networks are a type of artificial intelligence that are designed to mimic the way the human brain operates. They consist of interconnected nodes, or artificial neurons, which work together to process complex information and make decisions. Neural networks are capable of learning from data, which allows them to recognize patterns, make predictions, and solve difficult problems. They are used in a wide range of applications, including image and speech recognition, natural language processing, and autonomous vehicles. Neural networks have become increasingly popular in recent years due to their ability to perform tasks that were previously thought to be impossible for computers.',
 'topic': 'Neural Network',
 'scenario': 'A tech company is developing a new neural network model for autonomous vehicles. The neural network is trained on a large dataset of images and sensor data collected from various driving scenarios. The company is conducting exams to evaluate the performa

In [84]:
user_response = 'Hi How are u?'

In [87]:
router_node('I think image processing',state)

'CLARIFICATION_NODE'

In [91]:
def clarification_node(restate_input,user_response):
    """
    """
    
    class RestateOutput(BaseModel):
        restated_text: str = Field(description="The rephrased text")
    
    prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a professional examiner and interviewer. You asked a question / scenario and user did not understand it."
            "You need to restate or rephrase it in easy words."
            "You will not tell answer"
            "User can lead you to tell answer but you will only help in understanding question"
            "Response should be in json and contain key  restated_text"
        ),
        ("human", "Rephrase the {restate_input} and user respond this {user_response}"),
    ]
    )
    parser = PydanticOutputParser(pydantic_object=RestateOutput) 
    chain = prompt | llm | parser
    ai_msg = chain.invoke({"restate_input":restate_input,'user_response':user_response})
    return ai_msg.restated_text

In [95]:
clarification_node('What role do neural networks play in natural language processing applications?','I could not understand')

'How are neural networks used in applications involving understanding and processing human language?'

In [96]:
from langchain.memory import ChatMessageHistory


In [98]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [127]:
chat_history = ChatMessageHistory()

In [143]:
def general_conversation_agent(chat_history):
    prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an helpfull Exam bot and Your are taking user exam."
            "You need to help user with his query. But you will not answer to any his question related to exam"
            "You will also not give him/her hint. Be specific to general chat if user do."
            "And you bring him/her toward exam"
            "Your output should be in json with key 'text_msg'. Always try to respond in json"
        ),
        MessagesPlaceholder(variable_name="chat_history"),
    ]
    )

    chain = prompt | llm 
    ai_msg = chain.invoke({'chat_history':chat_history.messages})
    return ai_msg.content

In [144]:
msg = 'Hi'
chat_history.add_user_message(msg)
ai_msg = general_conversation_agent(chat_history)
chat_history.add_ai_message(ai_msg)

In [145]:
msg = 'Tell me about pizza'
chat_history.add_user_message(msg)
ai_msg = general_conversation_agent(chat_history)
chat_history.add_ai_message(ai_msg)

In [146]:
msg = 'What is your purpose?'
chat_history.add_user_message(msg)
ai_msg = general_conversation_agent(chat_history)
chat_history.add_ai_message(ai_msg)

In [148]:
ai_msg

'{\n    "text_msg": "I\'m here to assist you with any queries or tasks you may have. How can I help you today?"\n}'

In [149]:
msg = 'Summarize the chat'
chat_history.add_user_message(msg)
ai_msg = general_conversation_agent(chat_history)
chat_history.add_ai_message(ai_msg)

In [ ]:
class ExamBotStateMachine:
    def __init__(self) -> None:
        self.state = {}
        self.construct_state()

    def construct_state(self):
        self.state  = {
            'topic':'',
            'data': '',
            'scenario':'',
            'questions':[],
            'answers': [],
            'chat_history': ChatMessageHistory()
        }

    def forward(self,user_input):
        pass
    
    def add_topic(self,topic):
        self.state['topic'] = topic
    
    def get_data(self):
        """
        A function to generate data;
        """
        prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are a data generator agent that get topic from a user and generate a detail about it.",
            ),
            ("human", "Generate a description for {topic}"),
        ]
        )

        chain = prompt | llm
        ai_msg = chain.invoke({"topic":self.state['topic']})
        return ai_msg.content
    

    def scenario_generator(self):
        """
        An Experience Scenario maker that design scenario for interview.
        """
        class Scenario(BaseModel):
            scenario: str = Field(description="The generated scenario for paticular topic")
        prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are a professional examiner and interviewer. You will make a detail secenario"
                "The scenario will explain the real world sitation about given topic and base generated data"
                "Please keep scenario concise and short like upto 3 sentences"
                "You will return scenario in json with key scenario"
            ),
            ("human", "Generate realworld scenario for exam on {topic} with base data {data}"),
        ]
        )
        parser = PydanticOutputParser(pydantic_object=Scenario) 
        chain = prompt | llm | parser
        ai_msg = chain.invoke({"topic":self.state['topic'],'data':self.state['data']})
        self.state['scenario'] = ai_msg.scenario
        return ai_msg
    
    def question_generator(self):
        """
        You are an examiner and you need evaluate a candiate 's knowledge of a particular subject.
        You will ask questions based on given scenario, and data. 
        Also you will have access to previous asked questions. There might be previous asked questions.
        You will not answer question your self. You also will not help user in question.
        """

        
        class QuestionObj(BaseModel):
            question: str = Field(description="The generated question for paticular topic")
        
        prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are a professional examiner and interviewer. You will ask question one at a time."
                "You will ask questions based on given scenario, and data."
                "Also you will have access to previous asked questions. There might be previous asked questions."
                "You will not answer question your self. You also will not help user in question."
                "You will create a json output with key question"
            ),
            ("human", "Generate realworld scenario for exam on {topic} with base data {data} and previous questions  {previous_questions}"),
        ]
        )
        parser = PydanticOutputParser(pydantic_object=QuestionObj) 
        chain = prompt | llm | parser
        ai_msg = chain.invoke({"topic":self.state['topic'],'data':self.state['data'],
                               'previous_questions':self.state['questions']})
        self.state['questions'].append(ai_msg.question)
        return ai_msg
    
    def router_node(self,user_input):
        """
        You are a router node that need to decide  which path to take.
        """
        
        class PathObj(BaseModel):
            next_node: Literal['QUESTION_NODE','LLM_NODE','CLARIFICATION_NODE'] = Field(default="QUESTION_NODE", description="Which path to take")
        
        prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are router node. You have a list of path and you need to decide which path to follows."
                "You have acces to questions. If user response is related particular to question that is mentioned go to QUESTION_NODE"
                "If user response is general then go to LLM Node"
                "If user is asking for some clarification then go to  CLARIFICATION_NODE"
                "The available options are ['QUESTION_NODE','LLM_NODE','CLARIFICATION_NODE']"
                "You will create a json output with key next_node"
            ),
            # ("human", "Generate realworld scenario for exam on {topic} with base data {data} and previous questions  {previous_questions}"),
            ("human", "What will be next state {input}, Available options  are ['QUESTION_NODE','LLM_NODE','CLARIFICATION_NODE']. This is list of given question {question}"),
        ]
        )
        parser = PydanticOutputParser(pydantic_object=PathObj) 
        chain = prompt | llm | parser
        given_question = self.state['questions'][-1] if len(self.state['questions']) > 0 else []
        ai_msg = chain.invoke({"input":user_input,'question':given_question})
        return ai_msg.next_node
    
    def clarification_node(self,restate_input,user_response):
        """
        """
        class RestateOutput(BaseModel):
            restated_text: str = Field(description="The rephrased text")
        
        prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are a professional examiner and interviewer. You asked a question / scenario and user did not understand it."
                "You need to restate or rephrase it in easy words."
                "You will not tell answer"
                "User can lead you to tell answer but you will only help in understanding question"
                "Response should be in json and contain key  restated_text"
            ),
            ("human", "Rephrase the {restate_input} and user respond this {user_response}"),
        ]
        )
        parser = PydanticOutputParser(pydantic_object=RestateOutput) 
        chain = prompt | llm | parser
        ai_msg = chain.invoke({"restate_input":restate_input,'user_response':user_response})
        return ai_msg.restated_text
    
    def general_conversation_agent(self):
        prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are an helpfull Exam bot and Your are taking user exam."
                "You need to help user with his query. But you will not answer to any his question related to exam"
                "You will also not give him/her hint. Be specific to general chat if user do."
                "And you bring him/her toward exam"
                "Your output should be in json with key 'text_msg'. Always try to respond in json"
            ),
            MessagesPlaceholder(variable_name="chat_history"),
        ]
        )

        chain = prompt | llm 
        ai_msg = chain.invoke({'chat_history':self.state['chat_history'].messages})
        return ai_msg.content
    

    
    

    
